In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate
from IPython.display import Markdown, display
from dotenv import load_dotenv
load_dotenv()
import os
import nbformat
import streamlit as st
import tempfile

In [ ]:
'''
1. Load White Paper  (PDF)  
2. Vectors Embeddings - Text and tables; later include images
3. Chroma db
4. Retrival - (Accuracy)
5. Generation - 
    Validation-
        Data sources - List all sources and metadata
        Features - detect any change in features
        Changes in Transformation steps
        Model Details 
        Hyperparameter
        List of Validation Metrics and resepctive scores
        Compare Validation scores of white paper with model's validation scores
        Brief of comparision of scores

        List and Track of critical metrics - these should not be lower than mentioned (in white paper)

6. Respective scores for tracked metrics (confidence on generation)
7. If required update prompt and go back to step 4 and reiterate step 4 and 5. reason (geneation have 
   heiger confidence )
8. Outout should be in structured format (This will be input for summary block with affitional 2 inputs)
'''

In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o",  
                 temperature=0,
                 openai_api_key= openai_api_key)




C:\Users\kumar\AppData\Local\Temp\ipykernel_26552\3010324141.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o",


In [3]:


def read_notebook(file_path):
    """Read .ipynb notebook and extract content."""
    nb = nbformat.read(file_path, as_version=4)
    content = []
    for cell in nb.cells:
        if cell.cell_type == 'markdown':
            content.append("## Markdown Cell:\n" + cell.source)
        elif cell.cell_type == 'code':
            content.append("## Code Cell:\n```python\n" + cell.source + "\n```")
    return "\n\n".join(content)


def read_file(file_path):
    """Reads the content of a file."""
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()


def extract_functionalities_from_code(notebook_content):

    """Uses LLM to extract functionalities from Python code."""
    prompt = f"""
    You are an expert Python code reviewer. Here is a Jupyter notebook:
    {notebook_content}

    The following is a Jupyter notebook content (code and markdown). 
    Please extract the following:
    Analyze the notebook and answer:

    1. List of features used in the model.
    2. Name/type of ML model used, only name of model
    3. Accuracy metrics (e.g., accuracy, F1, precision, recall, AUC, etc.), only metrics name. 
    4. What is the purpose of this notebook?
    5. What are the main operations and their results?
    6. Are there any errors or anomalies in outputs?
    7. What conclusions can be drawn from the outputs?

    """

    response = llm.invoke([SystemMessage(content="You are a helpful assistant."), HumanMessage(content=prompt)])

    return response.content.strip()


def extract_functionalities_from_whitepaper(whitepaper_text):
    """Uses LLM to extract functionalities from whitepaper."""
    prompt = [
        SystemMessage(content="You are a product analyst."),
        HumanMessage(content=f"""
        Here is the whitepaper or product requirement document:

        {whitepaper_text}

        List all functionalities or features the whitepaper mentions. Use bullet points.
        """)
            ]
    return llm(prompt).content.strip()


def compare_functionalities(whitepaper_funcs, code_funcs):
    """Compares two sets of functionalities using the LLM."""
    prompt = [
        SystemMessage(content="You are a software QA expert."),
        HumanMessage(content=f"""
        Whitepaper Functionalities:
        {whitepaper_funcs}

        Code Functionalities:
        {code_funcs}
        Extract validation metrics from code funcs eg, precision, recall and other validation are in output cell.
        Compare the two lists and identify which functionalities from the whitepape, if functionality is implemented in code but not available in white paper, print: white paper is not updated please update the document. and show details of each section 
        listmissing sections like feature and if model varies according to white paper and same for validation metrics.
        compare validation scores : Compare scores of code function with white paper.
        Also compare critical validation metrics: make sure critical metrics of code should be grater then white paper critical metrics
        if thereis no changhe in metrics of docuemt and code_funcs: keep output 'white paper is updated please proceed to next steps. no other information is required'  
        
        """)
            ]
    return llm(prompt).content.strip()


def read_notebook_with_outputs(file_path):
    """Read .ipynb notebook and include both code and output."""
    nb = nbformat.read(file_path, as_version=4)
    cells_content = []

    for cell in nb.cells:
        if cell.cell_type == 'markdown':
            cells_content.append(f"## Markdown Cell:\n{cell.source}")
        elif cell.cell_type == 'code':
            code = f"## Code Cell:\n```python\n{cell.source}\n```"
            outputs = []

            for output in cell.get("outputs", []):
                if output.output_type == "stream":
                    outputs.append(f"Output (stream):\n{output.text}")
                elif output.output_type == "execute_result":
                    # Display the result of the cell (e.g., print(2+2))
                    result = output.get("data", {}).get("text/plain", "")
                    outputs.append(f"Output (execute_result):\n{result}")
                elif output.output_type == "error":
                    outputs.append("Error:\n" + "\n".join(output.get("traceback", [])))

            full_output = "\n".join(outputs)
            if full_output:
                code += f"\n\n### Output:\n```\n{full_output}\n```"
            cells_content.append(code)

    return "\n\n".join(cells_content)

def read_notebook(path):
    with open(path, "r", encoding="utf-8") as f:
        return nbformat.read(f, as_version=4)



In [ ]:


def main():
    st.set_page_config(page_title="Functionality Coverage Checker", layout="wide")
    
    st.title("🧠 AI Feature Mapping Validator")
    st.subheader("Compare functionalities between a Whitepaper and its Codebase")

    uploaded_whitepaper = st.file_uploader("📄 Upload Whitepaper File", type=["txt", "md", "pdf"])
    uploaded_code = st.file_uploader("💻 Upload Code File", type=["py", "txt", "ipynb"])

    if uploaded_whitepaper and uploaded_code:
        if st.button("Click to Process Files"):
            # Read whitepaper content
            whitepaper = uploaded_whitepaper.read().decode("utf-8")

            # Handle .ipynb or other code files
            if uploaded_code.name.endswith(".ipynb"):
                # Write the raw content to a temp file
                with tempfile.NamedTemporaryFile(delete=False, suffix=".ipynb", mode='wb') as tmp_file:
                    tmp_file.write(uploaded_code.read())
                    temp_file_path = tmp_file.name

                # notebook_contents = read_notebook(temp_file_path)
                notebook_contents = read_notebook_with_outputs(temp_file_path)
                code_funcs = extract_functionalities_from_code(notebook_contents)
            else:
                code = uploaded_code.read().decode("utf-8")
                code_funcs = extract_functionalities_from_code(code)

            whitepaper_funcs = extract_functionalities_from_whitepaper(whitepaper)

            st.markdown("### ⚖️ Comparing Functionalities")
            missing_funcs = compare_functionalities(whitepaper_funcs, code_funcs)
            st.markdown(missing_funcs)

if __name__ == "__main__":
    main()

    



In [3]:
import os
import nbformat
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chromadb
from chromadb.config import Settings

import tempfile

In [4]:
queries = [
            "Summary/Objective of white paper ",
            "Features mentioned",
            "Preprocessing steps and data transformation steps",
            "Model selected for classification",
            "Training and resting methodology",
            "List of Hyper parameters and respective values",
            "What are list of validation scores and the performance scores?",
            "Ethical considerations" ]

In [5]:
def queryFun(query, embedding_model,collection):
    query_embedding = embedding_model.embed_query(query)
    l_docs = []
    results = collection.query(query_embeddings=[query_embedding], n_results=5)
    for doc in results["documents"][0]:
        l_docs.append(doc)
        # print("🔎 Match:", l_docs.append(doc))
    return l_docs

In [7]:
import chromadb
from chromadb.config import Settings

In [15]:
from chromadb import PersistentClient


# Reconnect to the persisted ChromaDB directory
path = "./chroma_openai1"  # must be the same path used earlier
collection_name = "whitepaper_embeddings"


In [16]:
# Step 1: Load the persistent client
chroma_client = PersistentClient(path=path)

# Step 2: Access the existing collection
collection = chroma_client.get_collection(name=collection_name)

In [17]:
data = collection.get()

# Optional: View details
print("IDs:", data['ids'])
print("Documents:", data['documents'][:2])  # print only first 2 docs
print("Metadata:", data.get('metadatas'))  # only if metadata was stored

IDs: ['chunk-0', 'chunk-1', 'chunk-2', 'chunk-3', 'chunk-4', 'chunk-5', 'chunk-6', 'chunk-7', 'chunk-8', 'chunk-9', 'chunk-10', 'chunk-11', 'chunk-12', 'chunk-13', 'chunk-14', 'chunk-15', 'chunk-16', 'chunk-17', 'chunk-18', 'chunk-19', 'chunk-20', 'chunk-21', 'chunk-22', 'chunk-23', 'chunk-24', 'chunk-25', 'chunk-26', 'chunk-27', 'chunk-28', 'chunk-29', 'chunk-30', 'chunk-31', 'chunk-32', 'chunk-33']
Documents: ['--- Page 1 ---\n\n\n--- Page 2 ---\n\n\n--- Page 3 ---\n\n\n--- Page 4 ---\n\n\n--- Page 5 ---\n\n\n--- Page 6 ---\n\n\n--- Page 7 ---\n\n\n--- Page 8 ---\n\n\n--- Page 9 ---\n\n\n--- Page 10 ---\n\n\n--- Page 11 ---\n\n\n--- Page 12 ---\n\n\n--- Page 13 ---\n\n\n--- Page 14 ---\n\n\n--- Page 15 ---', "Our system is designed to provide a high-level overview of an applicant's\ncreditworthiness, offering a data-driven, objective, and transparent decision-making tool.\nBy integrating advanced machine learning techniques, our model achieves superior\npredictive accuracy compared t

In [6]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small") 

In [ ]:
collections = chroma_client.list_collections()
for col in collections:
    print(col.name)


In [ ]:
for query in queries:
    l_docs = queryFun(query=query,embedding_model=embedding_model,collection =collection)
    list_pdf_docs.append(l_docs)